<a href="https://colab.research.google.com/github/sebH1401/myarmory_analysis/blob/main/showcase_acq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Data Acquisition**



In [1]:
import pandas as pd
import requests
import numpy as np
import datetime
import pytz

In [7]:
with open('/content/lotus/log_list.txt', "r") as file:
    log_url_list = pd.Series(file.readlines())

In [11]:
def process_log(log_url: str) -> pd.DataFrame:
    pre_url = "https://dps.report/getJson?permalink="
    match_ids_boons = {
        718: "regen",
        740: "might",
        1187: "quick",
        30328: "alac",
        725: "fury"
        }
    try:
        response = requests.get(pre_url+log_url)
        log_data = response.json()
    except:
        return pd.DataFrame()
    dur_str = log_data["duration"]
    dur_str = dur_str.replace("m", "")
    dur_str = dur_str.replace("s", "")
    dur_str = dur_str.split()
    time = float(dur_str[0])*60+float(dur_str[1])
    timestamp_start = log_data["timeStartStd"]
    timestamp_end = log_data["timeEndStd"]
    encounter_name = log_data["fightName"]
    is_CM = log_data["isCM"]
    is_success = log_data["success"]
    player_stats = []
    for player in log_data["players"]:
        acc_name = player["account"]
        dps = player["dpsAll"][0]["dps"]
        dmg_taken = player["defenses"][0]["damageTaken"]
        downed = player["defenses"][0]["downCount"]
        if "deadDuration" in player["defenses"][0].keys():
            dead_duration = player["defenses"][0]["deadDuration"]/100
        else:
            dead_duration = 0
        cleanse = player["support"][0]["condiCleanse"]
        profession = player["profession"]
        stack_dist = player["statsAll"][0]["stackDist"]
        boons = player["buffUptimesActive"]
        boons_uptimes = {
            "might": None,
            "fury": None,
            "quick": None,
            "alac": None,
            "regen": None
            }

        for boon in boons:
            if boon["id"] in match_ids_boons.keys():
                boons_uptimes[match_ids_boons[boon["id"]]] = boon["buffData"][0]["uptime"]

        boons = player.get("groupBuffsActive")
        boons_generated = {
            "might": None,
            "fury": None,
            "quick": None,
            "alac": None,
            "regen": None
            }

        if boons is not None:
            for boon in boons:
                if boon["id"] in match_ids_boons.keys():
                    boons_generated[match_ids_boons[boon["id"]]] = boon["buffData"][0]["generation"]

        if player["statsTargets"][0][0]["connectedPowerCount"]==0:
            above90=0;
        else:
            above90 =100*player["statsTargets"][0][0]["connectedPowerAbove90HPCount"]/player["statsTargets"][0][0]["connectedPowerCount"]


        dataset= {
            "encounter": encounter_name,
            "is_cm": is_CM,
            "is_success": is_success,
            "time": time,
            "timestamp_start": timestamp_start,
            "timestamp_end": timestamp_end,
            "player": acc_name,
            "profession": profession,
            "dps": dps,
            "might up": boons_uptimes["might"],
            "fury up": boons_uptimes["fury"],
            "quick up": boons_uptimes["quick"],
            "alac up": boons_uptimes["alac"],
            "regen up": boons_uptimes["regen"],
            "might out": boons_generated["might"],
            "fury out": boons_generated["fury"],
            "quick out": boons_generated["quick"],
            "alac out": boons_generated["alac"],
            "regen out": boons_generated["regen"],
            "dmgTaken": dmg_taken,
            "stack_dist": stack_dist,
            "downed": downed,
            "dead_duration": dead_duration,
            "cleanses": cleanse,
            "above90%": above90,
            "log": log_url
            }
        player_stats.append(dataset)
    return pd.DataFrame(player_stats)

In [ ]:
process_logs = np.vectorize(process_log)

df = pd.concat(list(process_logs(log_url_list)))